In [2]:
import numpy as np
import tensorflow as tf
from curriculum_learning.models.classifier_model import ClassifierModel
from curriculum_learning import utils
import yaml
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [3]:
with open("models_hyperparameters.yaml", "r") as stream:
    models_hyperparameters = yaml.safe_load(stream)
    
with open("config_tests.yaml", "r") as stream:
    config_tests = yaml.safe_load(stream)

In [4]:
N_EPOCHS = 25
N_TRIALS = 50
BATCH_SIZE = 256

CONFIG = config_tests["assessment_proba_best"]

loss = tf.keras.losses.SparseCategoricalCrossentropy()

In [5]:
x1, y1 = utils.load_cifar_data("../data/cifar-10-batches-py/")
x1 /= 255

x, _, y, _ = train_test_split(x1, y1, train_size=0.4, random_state=42)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.1, random_state=42)

n_classes = len(np.unique(y))
train_size = x_train.shape[0]
train_size

16000

In [6]:
assessment_model = ClassifierModel(output_shape=n_classes, **models_hyperparameters["assessment_model"])

assessment_model.compile(loss=loss, metrics=["accuracy"])
assessment_model(x_train[:1])
assessment_model.load_weights("../models/assessment_model.weights.h5")

2024-03-24 14:41:07.211640: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Max
2024-03-24 14:41:07.211667: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2024-03-24 14:41:07.211674: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2024-03-24 14:41:07.211713: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-24 14:41:07.211732: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/Users/dbugajny/PycharmProjects/CurriculumLearning/venv/lib/python3.9/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'rmsprop

In [7]:
x_train_sorted = x_train[np.argsort(y_train)]
y_train_sorted = y_train[np.argsort(y_train)]
_, counts = np.unique(y_train_sorted, return_counts=True)

samples_proba = utils.calculate_proba(
    assessment_model, x_train_sorted, y_train_sorted, counts, CONFIG["negative_loss"]
)

2024-03-24 14:41:10.354312: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


In [9]:
model = ClassifierModel(output_shape=n_classes, **models_hyperparameters["test_model_1"])
model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])
model(x_train[0:1])
# model.save_weights("../models/default_model.weights.h5")
model.load_weights("../models/default_model.weights.h5")
model_weights = model.get_weights()

In [10]:
results = {}
model_scores = []

for _ in tqdm(range(N_TRIALS)):
    model.set_weights(model_weights)
        
    for i in range(N_EPOCHS):
        n_samples = int(np.tanh(4 * (i + 1) / N_EPOCHS) * train_size)
        
        samples_ids = utils.chose_samples(n_samples, samples_proba, CONFIG["order_type"])
        model.fit(
            x_train_sorted[samples_ids],
            y_train_sorted[samples_ids],
            # validation_data=(x_val, y_val),
            epochs=1,
            batch_size=BATCH_SIZE,
            verbose=0,
        )
        
        if CONFIG["progressive"]:
            samples_proba = utils.calculate_proba(
                model, x_train_sorted, y_train_sorted, counts, CONFIG["negative_loss"]
            )
            
    _, accuracy = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE, verbose=1)
    model_scores.append(accuracy)

    results["test_model_1"] = model_scores

  0%|          | 0/50 [00:00<?, ?it/s]

15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.5210 - loss: 2.2072


  2%|▏         | 1/50 [01:03<51:56, 63.59s/it]

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6143 - loss: 1.4198


  4%|▍         | 2/50 [01:56<45:57, 57.45s/it]

...

 96%|█████████▌| 48/50 [42:12<01:45, 52.52s/it]

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5797 - loss: 1.5248


 98%|█████████▊| 49/50 [43:04<00:52, 52.44s/it]

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6178 - loss: 1.2248


100%|██████████| 50/50 [43:59<00:00, 52.79s/it]


In [11]:
results

{'test_model_1': [0.522777795791626,
  0.6100000143051147,
  0.5322222113609314,
  0.621666669845581,
  0.5380555391311646,
  0.5563889145851135,
  0.6041666865348816,
  0.6302777528762817,
  0.5338888764381409,
  0.54666668176651,
  0.602222204208374,
  0.6155555844306946,
  0.5961111187934875,
  0.6127777695655823,
  0.5461111068725586,
  0.5911111235618591,
  0.6030555367469788,
  0.6102777719497681,
  0.5861111283302307,
  0.5677777528762817,
  0.5730555653572083,
  0.6000000238418579,
  0.597777783870697,
  0.5958333611488342,
  0.605555534362793,
  0.5874999761581421,
  0.5736111402511597,
  0.570277750492096,
  0.5952777862548828,
  0.6313889026641846,
  0.6133333444595337,
  0.5902777910232544,
  0.6127777695655823,
  0.5963888764381409,
  0.5691666603088379,
  0.644444465637207,
  0.5947222113609314,
  0.5680555701255798,
  0.6133333444595337,
  0.5930555462837219,
  0.49861112236976624,
  0.5447221994400024,
  0.5022222399711609,
  0.6166666746139526,
  0.6030555367469788,
  

In [ ]:
sum(results["test_model_1"]) / N_TRIALS